In [ ]:
import subprocess
import shlex
import os

from subprocess import PIPE
from pathlib import Path

In [ ]:
def submit_independent_spatially_explicit(repeats=10, walltime=1, speciation=0.000001, sample=0.00025, config='()', output='./STDIN'):
    walltime = "{:02d}".format(walltime)
    
    config = f"""
    #PBS -lselect={1}:ncpus={1}:mem={16}gb
    #PBS -J 0-{max(repeats - 1, 0)}
    #PBS -lwalltime={walltime}:00:00

    $HOME/necsim-rust/target-base/release/rustcoalescence simulate '(
        speciation: {speciation},
        sample: {sample},
        seed: '$(python3 -c "import random; print(random.getrandbits(64))")',

        algorithm: Independent{config},

        scenario: SpatiallyExplicit(
            habitat: "'$HOME'/necsim-rust/maps/madingley/fg0size12/habitat.tif",
            dispersal: "'$HOME'/necsim-rust/maps/madingley/fg0size12/dispersal.tif",
        ),

        reporters: [
            Plugin(
                library: "'$HOME'/necsim-rust/target-base/release/deps/libnecsim_plugins_common.so",
                reporters: [
                    Execution(),
                    Biodiversity(),
                ],
            ),
        ],
    )'
    """
    
    output = Path(output).resolve(strict=False)
    output.parent.mkdir(parents=True, exist_ok=True)
    
    cwd = os.getcwd()
    os.chdir(output.parent)
    
    # Submit the simulation batch
    result = subprocess.run(shlex.split(
        f"qsub -N {output.name}"
    ), check=True, input=config, stdout=PIPE, universal_newlines=True).stdout.strip()
    
    os.chdir(cwd)
    
    return result

In [ ]:
for delta_t in [0.1, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]:
    submit_independent_spatially_explicit(
        walltime=5, repeats=10, speciation=0.000001, sample=0.00025,
        config=f"(delta_t:{delta_t})", output=f"delta-t/pbs.{delta_t}",
    )

In [ ]:
for step_slice in [1, 2, 3, 4, 6, 8, 10, 12, 16, 20]:
    submit_independent_spatially_explicit(
        walltime=5, repeats=10, speciation=0.000001, sample=0.00025,
        config=f"(step_slice:{step_slice})", output=f"step-slice/pbs.{step_slice}",
    )

In [ ]:
for event_slice in [100, 316, 1000, 3162, 10000, 31623, 100000, 316228, 1000000]:
    submit_independent_spatially_explicit(
        walltime=5, repeats=10, speciation=0.000001, sample=0.00025,
        config=f"(parallelism_mode:Monolithic(event_slice:{event_slice}))", output=f"event-slice/pbs.{event_slice}",
    )

In [ ]:
for dedup_capacity in [1000, 3162, 10000, 31623, 100000, 316228, 1000000, 3162278, 10000000]:
    submit_independent_spatially_explicit(
        walltime=5, repeats=10, speciation=0.000001, sample=0.00025,
        config=f"(dedup_cache:Absolute(capacity:{dedup_capacity}))",
        output=f"dedup-capacity/pbs.{dedup_capacity}",
    )